## Import stuff

In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import shutil
import random
from PIL import Image
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
logger = tf.get_logger()
logger.setLevel(logging.ERROR)


from tensorflow.python.client import device_lib
device_lib.list_local_devices() 


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9685578220308911483
 xla_global_id: -1]

## Load training data from csv


In [6]:
csv = pd.read_csv("train.csv")

Y = csv["y"].tolist()
display(csv)
# load columns x1 to x13 as an array of vectors
X = np.array(csv[["x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","x12","x13"]].values)

X = np.array(X,  dtype=float)
Y = np.array(Y,  dtype=float)

# X = tf.cast(X, tf.float32)
# Y = tf.cast(Y, tf.float32)



,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,22.6,0.06724,0.0,3.24,0.0,0.460,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34
1,50.0,9.23230,0.0,18.10,0.0,0.631,6.216,100.0,1.1691,24.0,666.0,20.2,366.15,9.53
2,23.0,0.11425,0.0,13.89,1.0,0.550,6.373,92.4,3.3633,5.0,276.0,16.4,393.74,10.50
3,8.3,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77
4,21.2,0.05646,0.0,12.83,0.0,0.437,6.232,53.7,5.0141,5.0,398.0,18.7,386.40,12.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,16.6,0.67191,0.0,8.14,0.0,0.538,5.813,90.3,4.6820,4.0,307.0,21.0,376.88,14.81
146,32.9,0.01778,95.0,1.47,0.0,0.403,7.135,13.9,7.6534,3.0,402.0,17.0,384.30,4.45
147,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
148,11.0,7.36711,0.0,18.10,0.0,0.679,6.193,78.1,1.9356,24.0,666.0,20.2,96.73,21.52


## Create the model and compile it, with loss and optimizer functions

In [3]:
reg_params = [0.1, 1, 10, 100, 200]
res = np.zeros(5)

for i in range(5):

    reg = reg_params[i]
    all_RMSE = np.zeros(10)

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=10, shuffle=True)

    # K-fold Cross Validation model evaluation
    fold_no = 0
    for train, test in kfold.split(X, Y):

        # print(train.shape)
        # print(X.shape)
        # print(X[train].shape)
        # print(X[train])
        # print(test.shape)
        
        #,kernel_initializer='zeros'
        model = tf.keras.Sequential([
            layers.Dense(units=1,input_shape=[13],kernel_regularizer=tf.keras.regularizers.L2(reg),bias_regularizer=tf.keras.regularizers.L2(reg)),
        ])

        model.compile(tf.optimizers.Adam(learning_rate=0.01), loss='mse') #, metrics=['mse']

        history = model.fit(X[train], Y[train], epochs=3000, verbose=False) # 
        

        pred = model.predict(X[test])
        # print(pred)

        RMSE = mean_squared_error(Y[test], pred)**0.5

        
        all_RMSE[fold_no] = RMSE

        fold_no+=1

    print("Finished training for reg = ", reg)
    res[i] = np.mean(all_RMSE)
    # print("RMSE average with param ", reg,": ", np.average(all_RMSE))

print(res)


Finished training for reg =  0.1
Finished training for reg =  1
Finished training for reg =  10
Finished training for reg =  100
Finished training for reg =  200
[5.24685822 6.04009488 7.09867033 8.03853248 8.34648701]


# Export results

In [4]:
# write the RMSE to file "submit.csv"
with open("submit.csv", "w") as f:
    for i in res:
        f.write(str(i) + "\n")

